token_1.py is OK for milestone1

This file is still being improved...

In [1]:
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow
from tflearn.data_utils import VocabularyProcessor
import json
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time

enron_data_path = "../dataset/enron"

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def preprocess_one_set(data_path):
    res = []
    for root, dirs, files in os.walk(data_path):
        for file_name in files:
            cur_file_path = os.path.join(root, file_name)
            cur_txt = ""
            with open(cur_file_path) as cur_f:
                try:
                    cur_lines = cur_f.readlines()
                    for one_line in cur_lines:
                        one_line = one_line.strip('\n')
                        one_line = one_line.strip('\r')
                        cur_txt += one_line
                    res.append(cur_txt)
                except UnicodeDecodeError:
                    continue
    return res

def process_all_data():
    spam_list = []
    ham_list = []
    NUM_SET = 1
    for i in range(1, NUM_SET + 1):
        spam_path = enron_data_path + str(i) +  "/spam/"
        ham_path = enron_data_path + str(i) + "/ham/"
        cur_spam_list = preprocess_one_set(spam_path)
        cur_ham_list = preprocess_one_set(ham_path)
        spam_list += cur_spam_list
        ham_list += cur_ham_list
    return spam_list, ham_list

In [3]:
spam_list, ham_list = process_all_data()
emails_list = spam_list + ham_list
len_spam = len(spam_list)
len_ham = len(ham_list)
print("len of spam_list: ", len_spam)
print("len of ham_list: ", len_ham)
y = [1] * len_spam + [0] * len_ham

len of spam_list:  1487
len of ham_list:  3672


In [4]:
def feature_extraction_bagofwords(emails_list, num_features):
    # currently do not consider max_features
    tfidfv = TfidfVectorizer(
        decode_error = "ignore",
        analyzer = "word",
        stop_words = "english",
        smooth_idf = False,
        max_features = num_features,# at most 2010 features for two datasets
    )
    x = np.array(emails_list)
    x = tfidfv.fit_transform(x)
    x = x.toarray()
    # vocab_path = "../output/vocabulary_bagofwords.txt"
    # print("output the vocabulary to " + vocab_path + " ......\n")
    # with open(vocab_path, 'w') as f:
    #     f.write(json.dumps(tfidfv.vocabulary_))
    print("len of x: ", len(x))
    print("#features: ",len(x[0]))
    return x


def feature_extraction_vo(emails_list):
    vp = VocabularyProcessor(
        max_document_length = 1000,
        min_frequency = 1,
        vocabulary = None,
        tokenizer_fn = None
    )
    x = vp.fit_transform(emails_list)
    x = np.array(list(x))
    vocab_path = "../output/vocabulary_tf.txt"
    # with open(vocab_path, 'w') as f:
    #     f.write(json.dumps(vp.vocabulary_._mapping))
    print("len of x: ", len(x))
    return x

Support Vector Machines Algorithm

In [5]:
from sklearn import svm

def detection_svm(X_train, y_train, X_test, y_test):
    print("---Support Vector Machines Algorithm---")
    start_time = time.time()
    svm_c = svm.SVC()
    svm_c.fit(X_train, y_train)
    y_pred = svm_c.predict(X_test)
    end_time = time.time()

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    t = end_time - start_time
    res = [t, precision, recall, f1, accuracy]
    return res

K-nearest Neighbor Algorithm (KNN)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

def detection_knn(X_train, y_train, X_test, y_test):
    print("---K-nearest Neighbor Algorithm (KNN)---")
    start_time = time.time()
    knnc = KNeighborsClassifier()
    knnc.fit(X_train, y_train)
    y_pred = knnc.predict(X_test)
    end_time = time.time()

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    t = end_time - start_time
    res = [t, precision, recall, f1, accuracy]
    return res

Gaussian Naive Bayes

In [7]:
from sklearn.naive_bayes import GaussianNB

def detection_gnb(X_train, y_train, X_test, y_test):
    print("---Gaussian Naive Bayes---")
    start_time = time.time()
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred = gnb.predict(X_test)
    end_time = time.time()

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    t = end_time - start_time
    res = [t, precision, recall, f1, accuracy]
    return res

K-means Clustering

In [8]:
from sklearn.cluster import KMeans

def detection_kmeans(X_train, y_train, X_test, y_test, num_clusters):
    print("---K-means Clustering---")
    start_time = time.time()
    kmc = KMeans(n_clusters = num_clusters)
    kmc.fit(X_train)
    y_pred = kmc.predict(X_test)
    end_time = time.time()

    # print(y_test)
    # print(y_pred)
    y_processed = []
    for yi in y_pred:
        if yi == 0:
            y_processed.append(0)
        else:
            y_processed.append(1)
    precision = metrics.precision_score(y_test, y_processed)
    recall = metrics.recall_score(y_test, y_processed)
    f1 = metrics.f1_score(y_test, y_processed)
    accuracy = metrics.accuracy_score(y_test, y_processed)
    t = end_time - start_time
    res = [t, precision, recall, f1, accuracy]
    return res

Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier

def detection_rf(X_train, y_train, X_test, y_test):
    print("---Random Forest---")
    start_time = time.time()
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    end_time = time.time()

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    t = end_time - start_time
    res = [t, precision, recall, f1, accuracy]
    return res
    # print("precision: ", precision)
    # print("recall: ", recall)
    # print("f1: ", f1)
    # print("accuracy: ", accuracy)
    # print("\n")

Training and Testing

Bag of Words

In [10]:
NUM_FEATURE_ARR = [1000, 2000, 3000, 4000, 5000]
TEST_SIZE = 0.5

def bagofwords_split():
    X_train_bagofwords_arr = []
    X_test_bagofwords_arr = []
    y_train_bagofwords_arr = []
    y_test_bagofwords_arr = []

    for num_feature in NUM_FEATURE_ARR:
        x = feature_extraction_bagofwords(emails_list, num_feature)
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=0)
        X_train_bagofwords_arr.append(X_train)
        X_test_bagofwords_arr.append(X_test)
        y_train_bagofwords_arr.append(y_train)
        y_test_bagofwords_arr.append(y_test)
    return X_train_bagofwords_arr, X_test_bagofwords_arr, y_train_bagofwords_arr, y_test_bagofwords_arr

X_train_bagofwords_arr, X_test_bagofwords_arr, y_train_bagofwords_arr, y_test_bagofwords_arr = bagofwords_split()

def onetest():
    res = dict()
    for i in range(0, len(NUM_FEATURE_ARR)):
        #res_svm = detection_svm(X_train_bagofwords_arr[i], y_train_bagofwords_arr[i], X_test_bagofwords_arr[i], y_test_bagofwords_arr[i])
        res_knn = detection_knn(X_train_bagofwords_arr[i], y_train_bagofwords_arr[i], X_test_bagofwords_arr[i], y_test_bagofwords_arr[i])
        res_gnb = detection_gnb(X_train_bagofwords_arr[i], y_train_bagofwords_arr[i], X_test_bagofwords_arr[i], y_test_bagofwords_arr[i])
        res_kmeans = detection_kmeans(X_train_bagofwords_arr[i], y_train_bagofwords_arr[i], X_test_bagofwords_arr[i], y_test_bagofwords_arr[i], 2)
        res_rf = detection_rf(X_train_bagofwords_arr[i], y_train_bagofwords_arr[i], X_test_bagofwords_arr[i], y_test_bagofwords_arr[i])
        temp = [res_knn, res_gnb, res_kmeans, res_rf]
        res[NUM_FEATURE_ARR[i]] = temp
    return res

res = onetest()
print(res)
    

len of x:  5159
#features:  1000
len of x:  5159
#features:  2000
len of x:  5159
#features:  3000
len of x:  5159
#features:  4000
len of x:  5159
#features:  5000
---K-nearest Neighbor Algorithm (KNN)---
---Gaussian Naive Bayes---
---K-means Clustering---
---Random Forest---
---K-nearest Neighbor Algorithm (KNN)---
---Gaussian Naive Bayes---
---K-means Clustering---
---Random Forest---
---K-nearest Neighbor Algorithm (KNN)---
---Gaussian Naive Bayes---
---K-means Clustering---
---Random Forest---
---K-nearest Neighbor Algorithm (KNN)---
---Gaussian Naive Bayes---
---K-means Clustering---
---Random Forest---
---K-nearest Neighbor Algorithm (KNN)---
---Gaussian Naive Bayes---
---K-means Clustering---
---Random Forest---
{1000: [[0.867828369140625, 0.9243243243243243, 0.9144385026737968, 0.9193548387096774, 0.9534883720930233], [0.07200360298156738, 0.8893129770992366, 0.9344919786096256, 0.9113428943937418, 0.9472868217054263], [0.803351879119873, 0.0, 0.0, 0.0, 0.5891472868217055], [0

In [11]:
res_svm = detection_svm(X_train, y_train, X_test, y_test)
res_knn = detection_knn(X_train, y_train, X_test, y_test)
res_gnb = detection_gnb(X_train, y_train, X_test, y_test)
res_kmeans = detection_kmeans(X_train, y_train, X_test, y_test, 2)
res_rf = detection_rf(X_train, y_train, X_test, y_test)

NameError: name 'X_train' is not defined

VO

In [ ]:
TEST_SIZE = 0.4

x = feature_extraction_vo(emails_list)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=0)

len of x:  5159


In [ ]:
res_svm = detection_svm(X_train, y_train, X_test, y_test)
res_knn = detection_knn(X_train, y_train, X_test, y_test)
res_gnb = detection_gnb(X_train, y_train, X_test, y_test)
res_kmeans = detection_kmeans(X_train, y_train, X_test, y_test, 2)
res_rf = detection_rf(X_train, y_train, X_test, y_test)

---Support Vector Machines Algorithm---
---K-nearest Neighbor Algorithm (KNN)---
---Gaussian Naive Bayes---
---K-means Clustering---
ss:  {0, 1}
---Random Forest---


In [ ]:
print(res_svm)
print(res_knn)
print(res_gnb)
print(res_kmeans)
print(res_rf)

[8.802137851715088, 0.8040201005025126, 0.27586206896551724, 0.41078305519897307, 0.7776162790697675]
[0.7527022361755371, 0.5408560311284046, 0.2396551724137931, 0.3321385902031064, 0.7291666666666666]
[0.06689810752868652, 0.4439252336448598, 0.16379310344827586, 0.23929471032745595, 0.7073643410852714]
[1.0448384284973145, 1.0, 0.020689655172413793, 0.040540540540540536, 0.7248062015503876]
[1.4990005493164062, 0.8758389261744967, 0.45, 0.5945330296127562, 0.8275193798449613]
